<a href="https://colab.research.google.com/github/RepoRecipee/IIIT-PROJECTS-AND-LABS/blob/main/Saffron_AI_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend Assignment - 1

# Step 1: Import Required Libraries

In [1]:
# Step 1: Import Required Libraries
!pip install mstarpy

import pandas as pd
import json
import datetime
from mstarpy import Funds


# Step 2: Upload the JSON File

In [2]:
# Step 2: Upload the JSON File
from google.colab import files

# This will prompt you to upload the JSON file
uploaded = files.upload()

# You can now access the file as follows
import json

# Assuming the uploaded file is named 'transaction_detail.json'
file_path = 'transaction_detail.json'  # Change this if your filename is different

with open(file_path, 'r') as f:
    data = json.load(f)  # Load the JSON data


Saving transaction_detail.json to transaction_detail.json


# Step 3: Load and Parse the JSON File

In [3]:
# Load the JSON file and print its structure
import json

file_path = 'transaction_detail.json'  # Ensure this matches the uploaded file name

with open(file_path, 'r') as f:
    data = json.load(f)  # Load the JSON data

# Print the keys in the loaded JSON data
print(json.dumps(data, indent=4))  # This will pretty-print the JSON data


{
    "fromDate": "01-Jan-2018",
    "data": [
        {
            "dtSummary": [
                {
                    "kycStatus": "3",
                    "rtaCode": "CAMS",
                    "decimalAmount": "2",
                    "scheme": "02GZ",
                    "costValue": "10000",
                    "closingBalance": "291.047",
                    "lastTrxnDate": "20-JAN-2020",
                    "taxStatus": "01",
                    "brokerCode": "INA100009859",
                    "lastNavDate": "30-AUG-2024",
                    "email": "dummy@gmail.com",
                    "openingBal": "0.000",
                    "nav": "67.58",
                    "schemeName": "Aditya Birla Sun Life ELSS Tax Saver Fund- (ELSS U/S 80C of IT ACT) - Growth-Direct Plan ",
                    "marketValue": "19668.96",
                    "isDemat": "N",
                    "decimalNav": "2",
                    "assetType": "EQUITY",
                    "amc": "B",
         

# Step 4: Process Transactions with FIFO Logic

In [4]:
import json

# File name of the uploaded JSON
file_name = 'transaction_detail.json'  # This should match the uploaded file name

# Load the JSON data
with open(file_name, 'r') as f:
    data = json.load(f)  # Load the JSON data

# Print the structure of the loaded JSON data to understand its format
print(json.dumps(data, indent=4))  # This will help us understand the structure

# Load Transactions Function
def load_transactions(data):
    # Extract transactions from the data structure
    return data['data'][0]['dtTransaction']  # Assuming we want the first entry's transactions

# Load your transactions
transactions = load_transactions(data)  # Load transactions here

# Calculate Portfolio
def calculate_portfolio(transactions):
    portfolio = {}

    for txn in transactions:
        folio = txn['folio']
        isin = txn['isin']
        units = float(txn['trxnUnits'])
        purchase_price = float(txn['purchasePrice'])

        # Update portfolio dictionary
        if (folio, isin) not in portfolio:
            portfolio[(folio, isin)] = {
                'net_units': 0,
                'acquisition_cost': 0,
                'current_value': 0
            }

        # Apply FIFO logic
        if units > 0:  # Purchase
            portfolio[(folio, isin)]['net_units'] += units
            portfolio[(folio, isin)]['acquisition_cost'] += units * purchase_price
        else:  # Sale
            units_to_sell = abs(units)
            while units_to_sell > 0 and portfolio[(folio, isin)]['net_units'] > 0:
                if portfolio[(folio, isin)]['net_units'] >= units_to_sell:
                    portfolio[(folio, isin)]['net_units'] -= units_to_sell
                    portfolio[(folio, isin)]['acquisition_cost'] -= units_to_sell * purchase_price
                    units_to_sell = 0
                else:
                    units_to_sell -= portfolio[(folio, isin)]['net_units']
                    portfolio[(folio, isin)]['acquisition_cost'] = 0
                    portfolio[(folio, isin)]['net_units'] = 0

    return portfolio

# Calculate the portfolio based on transactions
portfolio = calculate_portfolio(transactions)

# Display the results
for (folio, isin), values in portfolio.items():
    print(f"Folio: {folio}, ISIN: {isin}, Net Units: {values['net_units']:.2f}, Acquisition Cost: {values['acquisition_cost']:.2f}")


{
    "fromDate": "01-Jan-2018",
    "data": [
        {
            "dtSummary": [
                {
                    "kycStatus": "3",
                    "rtaCode": "CAMS",
                    "decimalAmount": "2",
                    "scheme": "02GZ",
                    "costValue": "10000",
                    "closingBalance": "291.047",
                    "lastTrxnDate": "20-JAN-2020",
                    "taxStatus": "01",
                    "brokerCode": "INA100009859",
                    "lastNavDate": "30-AUG-2024",
                    "email": "dummy@gmail.com",
                    "openingBal": "0.000",
                    "nav": "67.58",
                    "schemeName": "Aditya Birla Sun Life ELSS Tax Saver Fund- (ELSS U/S 80C of IT ACT) - Growth-Direct Plan ",
                    "marketValue": "19668.96",
                    "isDemat": "N",
                    "decimalNav": "2",
                    "assetType": "EQUITY",
                    "amc": "B",
         

# Step 5: Fetch Current NAV for Each Scheme

In [5]:
# Step 5: Fetch Current NAV for Each Scheme

def fetch_current_nav(isin):
    fund = Funds(term=isin, country="in")
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=365)
    history = fund.nav(start_date=start_date, end_date=end_date, frequency="daily")
    current_nav = history['nav'].iloc[-1]  # Latest NAV
    return current_nav


# Step 6: Calculate Total Portfolio Value and Gain

In [6]:
import json
import datetime
from mstarpy import Funds  # Assuming `mstarpy` works for you

# Load the JSON file
file_name = 'transaction_detail.json'  # Make sure this matches your file

# Load the JSON data
with open(file_name, 'r') as f:
    data = json.load(f)  # Load the JSON data

# Load Transactions Function
def load_transactions(data):
    return data['data'][0]['dtTransaction']  # Adjust if necessary based on your file

# Load your transactions
transactions = load_transactions(data)  # Load transactions here

# Fetch Current NAV for Each Scheme
def fetch_current_nav(isin):
    try:
        fund = Funds(term=isin, country="in")
        end_date = datetime.datetime.now()
        start_date = end_date - datetime.timedelta(days=365)
        history = fund.nav(start_date=start_date, end_date=end_date, frequency="daily")

        # Check if history is returned as a list and contains entries
        if isinstance(history, list) and len(history) > 0:
            current_nav = history[-1]['nav']  # Assuming 'nav' is a valid key, adjust if necessary
            return current_nav
        else:
            print(f"No NAV data found for ISIN {isin}")
            return 0  # Return 0 if no NAV data found
    except Exception as e:
        print(f"Error fetching NAV for ISIN {isin}: {e}")
        return 0  # Return 0 if an error occurs

# Calculate Portfolio
def calculate_portfolio(transactions):
    portfolio = {}

    for txn in transactions:
        folio = txn['folio']
        isin = txn['isin']
        units = float(txn['trxnUnits'])
        purchase_price = float(txn['purchasePrice'])

        # Update portfolio dictionary
        if (folio, isin) not in portfolio:
            portfolio[(folio, isin)] = {
                'net_units': 0,
                'acquisition_cost': 0
            }

        # Apply FIFO logic
        if units > 0:  # Purchase
            portfolio[(folio, isin)]['net_units'] += units
            portfolio[(folio, isin)]['acquisition_cost'] += units * purchase_price
        else:  # Sale
            units_to_sell = abs(units)
            while units_to_sell > 0 and portfolio[(folio, isin)]['net_units'] > 0:
                if portfolio[(folio, isin)]['net_units'] >= units_to_sell:
                    portfolio[(folio, isin)]['net_units'] -= units_to_sell
                    portfolio[(folio, isin)]['acquisition_cost'] -= units_to_sell * purchase_price
                    units_to_sell = 0
                else:
                    units_to_sell -= portfolio[(folio, isin)]['net_units']
                    portfolio[(folio, isin)]['acquisition_cost'] = 0
                    portfolio[(folio, isin)]['net_units'] = 0

    return portfolio

# Calculate the portfolio based on transactions
portfolio = calculate_portfolio(transactions)

# Calculate total value and gain
def calculate_portfolio_value_and_gain(portfolio):
    total_value = 0
    total_gain = 0

    for (folio, isin), values in portfolio.items():
        current_nav = fetch_current_nav(isin)

        # Calculate the current value and gain
        current_value = values['net_units'] * current_nav
        acquisition_cost = values['acquisition_cost']

        total_value += current_value
        total_gain += (current_value - acquisition_cost)

        print(f"Folio: {folio}, ISIN: {isin}, Net Units: {values['net_units']:.2f}, Current NAV: ₹{current_nav:.2f}, Current Value: ₹{current_value:.2f}, Acquisition Cost: ₹{acquisition_cost:.2f}")

    return total_value, total_gain

# Calculate total value and gain
total_value, total_gain = calculate_portfolio_value_and_gain(portfolio)

# Display total value and gain in Indian Rupees (₹)
print(f"\nTotal Portfolio Value: ₹{total_value:.2f}")
print(f"Total Portfolio Gain: ₹{total_gain:.2f}")


Folio: 1039101537, ISIN: INF209K01UN8, Net Units: 291.05, Current NAV: ₹67.25, Current Value: ₹19572.91, Acquisition Cost: ₹10000.02
Folio: 22399192, ISIN: INF090I01JR0, Net Units: 0.00, Current NAV: ₹78.76, Current Value: ₹0.00, Acquisition Cost: ₹223.68
Folio: 2684715, ISIN: INF194K01Y29, Net Units: 147.12, Current NAV: ₹177.93, Current Value: ₹26176.91, Acquisition Cost: ₹11499.48
Folio: 17154744, ISIN: INF179K01WN9, Net Units: 989.85, Current NAV: ₹768.47, Current Value: ₹760664.88, Acquisition Cost: ₹579972.69
Folio: 11127350, ISIN: INF174KA1FQ7, Net Units: 5114.31, Current NAV: ₹16.95, Current Value: ₹86712.63, Acquisition Cost: ₹49997.50
Folio: 6905767, ISIN: INF174K01LT0, Net Units: 176.11, Current NAV: ₹152.63, Current Value: ₹26880.00, Acquisition Cost: ₹7999.97
Folio: 23711519, ISIN: INF200KA1507, Net Units: 0.00, Current NAV: ₹42.10, Current Value: ₹0.00, Acquisition Cost: ₹-25364.27
Folio: 32895783, ISIN: INF200K01TP4, Net Units: 1515.70, Current NAV: ₹270.10, Current Valu

# Step 7: Display Total Portfolio Value and Gain

In [7]:
# Display total value and gain in Indian Rupees (₹)
print(f"\nTotal Portfolio Value: ₹{total_value:.2f}")
print(f"Total Portfolio Gain: ₹{total_gain:.2f}")


Total Portfolio Value: ₹4569440.06
Total Portfolio Gain: ₹1193223.74
